In [ ]:
!pip install requests beautifulsoup4 transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.5 MB/s eta 0:00:00


In [2]:
import requests
import warnings
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor

# Set of visited URLs to avoid duplicate scraping
visited_urls = set()

# Set of seen document content to avoid repeating output
seen_doc_content = set()

def scrape_page(url, max_depth=1, current_depth=0):
    # Get the HTML content of the URL
    response = requests.get(url)
    html_content = response.content

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the main content section of the page
    main_content = soup.find('div', {'class': 'section'})

    if main_content is None:
        # No main content section found, skip scraping this page
        return

    # Extract the relevant content from the main content section
    doc_content = '\n'.join(element.text.strip() for element in main_content.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'p']))

    # Display the extracted content to the user
    if doc_content not in seen_doc_content:
        print(doc_content)
        seen_doc_content.add(doc_content)

    # Find all the links within the page
    urls = [urljoin(url, link.get('href')) for link in soup.find_all('a') if link.get('href') and link.parent.name != 'div' and link.parent.get('class') != 'wy-nav-content' and link['href'].startswith(url)]

    # Recursively scrape each link
    with ThreadPoolExecutor() as executor:
        futures = []
        for link in urls:
            if current_depth < max_depth and link not in visited_urls:
                visited_urls.add(link)
                future = executor.submit(scrape_page, link, max_depth, current_depth + 1)
                futures.append(future)

        # Wait for all tasks to complete
        for future in futures:
            future.result()

# Get the initial URL
initial_url = input("Enter the URL: ")

# Start scraping from the initial URL
scrape_page(initial_url, max_depth=3)

# Open the text file for writing
with open('output.txt', 'w') as file:
    file.write('\n'.join(seen_doc_content))


Enter the URL: https://python-adv-web-apps.readthedocs.io/en/latest/learn_python.html
Starting Python¶
To begin learning Python, read chapters 1 and 2 in Sweigart’s Automate the Boring Stuff with Python (second edition). Some scripts in this folder are based on Sweigart’s — naturally, he has more examples than only these. At his website, there’s a link to download files used in the book.
Python scripts for this section
Install Python¶
If you haven’t installed Python yet, I made a Google doc for that:
Getting Started with Python 3
Two other helpful docs are linked on the CONTENTS page (under “Outside Resources”):
Contents page
Interactively learning Python¶
Start by running simple expressions and statements in the interactive Python interpreter (interactive shell). Open Terminal (Mac) or Command Prompt (Windows). Type python3 (Mac) or python (Windows) at the prompt to start the interpreter. When you see >>>, you’re in the shell. This will help you get comfortable with variables and basi

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

with open("output.txt", "r") as file:
    text = file.read()

inputs = tokenizer([text], truncation=True, padding=True, max_length=1024, return_tensors="pt")
summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=100, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)

with open("summ.txt", "w") as file:
    file.write(summary)


Start by running simple expressions and statements in the interactive Python interpreter. If you don’t type the code yourself, you will not learn it. After you’ve practiced basic expressions and variables, you need to learn how to save a script as a file and run it.


In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("codeparrot/codeparrot-small-code-to-text")
model = AutoModelForCausalLM.from_pretrained("codeparrot/codeparrot-small-code-to-text")

# Ask the user for input
user_choice = input("Do you want to use the prompt from the text file (summ.txt)? (y/n): ")

if user_choice.lower() == "y":
    # Read the prompt from summ.txt
    with open("summ.txt", "r") as file:
        prompt = file.read()
else:
    # Ask the user to input the prompt
    prompt = input("Enter your prompt: ")

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Generate code
output = model.generate(input_ids, max_length=100, num_return_sequences=1, num_beams=5)

# Decode the generated code
generated_code = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the generated code
print(generated_code)


Do you want to use the prompt from the text file (summ.txt)? (y/n): n
Enter your prompt: write a python function  to sum two numbers


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


write a python function  to sum two numbers:
End of explanation
"""


def sum_two_numbers(a, b):
    return a + b

print(sum_two_numbers(1, 2))
print(sum_two_numbers(1, 3))
print(sum_two_numbers(1, 4))
print(sum_two_numbers(1, 5))
print(sum_two_numbers(1, 6))
